In [148]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [172]:
df = pd.read_csv("../data/adult.csv")
filters = (
        (df["workclass"] != "?") & (df["education"] != "?") &
        (df["marital-status"] != "?") & (df["occupation"] != "?") &
        (df["relationship"] != "?") & (df["race"] != "?") &
        (df["sex"] != "?") & (df["native-country"] != "?") &
        ((df["salary"] == ">50K") == (df["salary K$"] > 50)) &
        ((df["salary"] == "<=50K") == (df["salary K$"] <= 50))        
)
df = df[filters]
df

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
32558,32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40
32559,32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38


# Task 1
Print the count of men and women in the dataset.

In [150]:
df["sex"].value_counts()

Male      20380
Female     9782
Name: sex, dtype: int64

# Task 2
Find the average age of men in dataset

In [151]:
mans = df[df["sex"] == "Female"]
mans["age"].mean()

36.8834594152525

# Task 3
Get the percentage of people from Poland (native-country)

In [152]:
percent_from_poland = df[df["native-country"] == "Poland"].count() / df.shape[0] * 100
percent_from_poland

Unnamed: 0        0.185664
age               0.185664
workclass         0.185664
education         0.185664
marital-status    0.185664
occupation        0.185664
relationship      0.185664
race              0.185664
sex               0.185664
hours-per-week    0.185664
native-country    0.185664
salary            0.185664
salary K$         0.185664
dtype: float64

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [153]:
people_more_50k_mean = df[df["salary"] == ">50K"].mean(numeric_only=True)
people_more_50k_std = df[df["salary"] == ">50K"].std(numeric_only=True)

people_to_50k_mean = df[df["salary"] == "<=50K"].mean(numeric_only=True)
people_to_50k_std = df[df["salary"] == "<=50K"].std(numeric_only=True)

people_more_50k_mean, people_more_50k_std, people_to_50k_mean, people_to_50k_std 

(Unnamed: 0        16388.867741
 age                  43.959110
 hours-per-week       45.706580
 salary K$           200.584310
 dtype: float64,
 Unnamed: 0        9440.543257
 age                 10.269633
 hours-per-week      10.736987
 salary K$           87.300441
 dtype: float64,
 Unnamed: 0        16253.376402
 age                  36.608060
 hours-per-week       39.348592
 salary K$            32.005518
 dtype: float64,
 Unnamed: 0        9364.352080
 age                 13.464631
 hours-per-week      11.950774
 salary K$           10.119256
 dtype: float64)

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [154]:
education_filter = (
    (df["education"] != "Bachelors") & (df["education"] != "Prof-school") &
    (df["education"] != "Assoc-acdm") & (df["education"] != "Assoc-voc") &
    (df["education"] != "Master") & (df["education"] != "Doctorate")
)
without_high_education = df[education_filter]
without_high_education[without_high_education["salary"] == ">50K"]

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
7,7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307
8,8,31,Private,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,50,United-States,>50K,330
10,10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K,116
19,19,43,Self-emp-not-inc,Masters,Divorced,Exec-managerial,Unmarried,White,Female,45,United-States,>50K,151
55,55,43,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,>50K,341
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32518,32518,57,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,116
32519,32519,46,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,48,United-States,>50K,239
32554,32554,53,Private,Masters,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,>50K,103
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [155]:
df.groupby("education").describe()

Unnamed: 0                                                      \
                  count          mean          std    min      25%      50%   
education                                                                     
10th              820.0  16099.841463  9323.630350  219.0  7860.75  16110.5   
11th             1048.0  16230.614504  9435.727749    3.0  7773.25  16491.5   
12th              377.0  17524.734748  9160.712573  415.0  9895.00  18630.0   
1st-4th           151.0  17502.503311  9788.829534  416.0  9009.00  17445.0   
5th-6th           288.0  16845.131944  9273.562959   56.0  9557.75  17173.0   
7th-8th           557.0  16283.675045  9328.136399   15.0  8014.00  16464.0   
9th               455.0  16306.367033  9438.104861    6.0  7638.50  16734.0   
Assoc-acdm       1008.0  16667.503968  9503.534369   13.0  8176.00  16897.5   
Assoc-voc        1307.0  16520.351186  9429.290747   48.0  8453.50  16722.0   
Bachelors        5044.0  16187.880452  9355.163922    0.0  8265.75  16108.0   
Doctorate         375.0  16600.130667  9141.070844   20.0  9067.50  16573.0   
HS-grad          9840.0  16166.445935  9338.758443    2.0  8173.75  16058.5   
Masters          1627.0  16341.271051  9497.768306    5.0  8094.00  16134.0   
Preschool          45.0  18268.377778  9824.639913  224.0  8775.00  21388.0   
Prof-school       542.0  16724.850554  9575.872379   52.0  8325.50  17072.0   
Some-college     6678.0  16266.526804  9414.721651   10.0  7986.25  16238.5   

                                    age             ... hours-per-week        \
                   75%      max   count       mean  ...            75%   max   
education                                           ...                        
10th          24230.75  32551.0   820.0  37.897561  ...           40.0  99.0   
11th          24275.25  32522.0  1048.0  32.363550  ...           40.0  99.0   
12th          25040.00  32535.0   377.0  32.013263  ...           40.0  99.0   
1st-4th       27132.50  32425.0   151.0  44.622517  ...           40.0  96.0   
5th-6th       25550.25  32334.0   288.0  41.649306  ...           40.0  75.0   
7th-8th       24473.00  32517.0   557.0  47.631957  ...           40.0  99.0   
9th           24516.00  32501.0   455.0  40.303297  ...           40.0  99.0   
Assoc-acdm    24918.25  32556.0  1008.0  37.286706  ...           45.0  99.0   
Assoc-voc     24410.50  32552.0  1307.0  38.246366  ...           45.0  99.0   
Bachelors     24284.75  32538.0  5044.0  38.641554  ...           50.0  99.0   
Doctorate     24299.00  32532.0   375.0  47.130667  ...           55.0  99.0   
HS-grad       24203.00  32560.0  9840.0  38.640955  ...           44.0  99.0   
Masters       24705.00  32554.0  1627.0  43.740012  ...           50.0  99.0   
Preschool     25652.00  32432.0    45.0  41.288889  ...           40.0  75.0   
Prof-school   25123.25  32548.0   542.0  44.249077  ...           55.0  99.0   
Some-college  24486.75  32555.0  6678.0  36.135370  ...           45.0  99.0   

             salary K$                                                     \
                 count        mean         std   min   25%    50%     75%   
education                                                                   
10th             820.0   43.912195   47.890388  15.0  25.0   33.0   43.00   
11th            1048.0   41.527672   46.846283  15.0  24.0   32.0   42.00   
12th             377.0   42.761273   47.213033  15.0  24.0   33.0   43.00   
1st-4th          151.0   37.556291   34.351446  15.0  24.0   31.0   41.50   
5th-6th          288.0   39.857639   42.081933  15.0  23.0   33.0   43.00   
7th-8th          557.0   42.095153   47.388376  15.0  24.0   32.0   42.00   
9th              455.0   43.028571   50.022784  15.0  24.5   33.0   42.00   
Assoc-acdm      1008.0   75.566468   87.291460  15.0  26.0   38.0   61.25   
Assoc-voc       1307.0   76.102525   86.446091  15.0  27.0   39.0   64.00   
Bachelors       5044.0  102.832276  101.376121  15.0  29.0   44.0 

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [169]:
married_filter = (
        (df["sex"] == "Female") & (df["marital-status"].str.contains("Married"))
)
unmarried_filter = (
        (df["sex"] == "Female") & (~df["marital-status"].str.contains("Married"))
)

married = df[married_filter]
unmarried = df[unmarried_filter]

salary_df = pd.DataFrame({"married": married["salary"].value_counts(), "unmarried": unmarried["salary"].value_counts()})
salary_df

,married,unmarried
<=50K,964,7706
>50K,717,395


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [157]:
df[df["hours-per-week"] == df["hours-per-week"].max()].count()

Unnamed: 0        78
age               78
workclass         78
education         78
marital-status    78
occupation        78
relationship      78
race              78
sex               78
hours-per-week    78
native-country    78
salary            78
salary K$         78
dtype: int64

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [158]:
df.corr()

,Unnamed: 0,age,hours-per-week,salary K$
Unnamed: 0,1.000000,-0.001126,-0.001890,0.000129
age,-0.001126,1.000000,0.101599,0.208203
hours-per-week,-0.001890,0.101599,1.000000,0.196378
salary K$,0.000129,0.208203,0.196378,1.000000
